In [3]:
import os
import sys
#import PySpin
import numpy as np
import ipywidgets as widgets

# Required for advanced widgets
from traitlets import traitlets

# Required for streaming
import cv2
import time
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show, push_notebook

# For async operation
import threading
import asyncio

In [4]:
streaming_toggle = widgets.ToggleButton(
                        value=False,
                        description='Start Streaming',
                        disabled=False,
                        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                        tooltip='Streaming',
                        icon='play'
                    )
sinference_switch = widgets.ToggleButton(
                        value=False,
                        description='Inference Enable',
                        disabled=False,
                        button_style='', # '', 'info', 'warning', 'danger' or ''
                        tooltip='Turn on Inference',
                        icon='binoculars'
                      )
inference_mode = widgets.ToggleButtons(
                        options=['Detection', 'Classification'],
                        description='Mode:',
                        disabled=False,
                        button_style='', # 'success', 'info', 'warning', 'danger' or ''
                        tooltips=['This network determines the  most likely class given a set of\
                                  predetermined, trained options. Object detection can also provide a\
                                  location within the image (in the form of a "bounding box" surrounding the\
                                  class), and can detect multiple objects',
                                  'This network determines the best option from a list of predetermined options\
                                  ; the camera gives a percentage that determines the likelihood of the currently\
                                  perceived image being one of the classes it has been trained to recognize.'],
                      )

inference_checkbox = widgets.Checkbox(
                    value=False,
                    description='Inference',
                    disabled=False
                )


bounding_box_checkbox = widgets.Checkbox(
                    value=False,
                    description='Bounding Box',
                    disabled=False
                )

In [40]:
""" INFERENCE HELPERs """



' INFERENCE HELPERs '

In [41]:
def show_init_screen():
    image_data = cv2.imread("welcome.jpeg")
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGBA)
    image_data = cv2.flip(image_data, 0)
    myImage.data_source.data['image']=[image_data]
    push_notebook()
    
def get_box_count(image):
    chunk_data = image.GetChunkData()
    box_result = chunk_data.GetInferenceBoundingBoxResult()
    box_count = box_result.GetBoxCount()
    return box_count

def get_box_result(image):
    chunk_data = image.GetChunkData()
    box_result = chunk_data.GetInferenceBoundingBoxResult()
    return box_result

def render_bounding_box(frame, box_count, box_result):
    result_frame = cv2.rectangle(frame, (384,0),(510,128), (0,255,0),3)
    result_frame = cv2.rectangle(frame, (500,0),(700,500), (0,255,0),3)
    return result_frame
    
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

image_data = cv2.imread('welcome.jpeg')
image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGBA)
image_data = cv2.flip(image_data, 0)
image_data = image_resize(image_data, height = 400)
width = image_data.shape[1]
height = image_data.shape[0]
streaming_window = figure(x_range=(0,width), y_range=(0,height), \
                                       output_backend="webgl", width=width, height=height)
myImage = streaming_window.image_rgba(image=[image_data], x=0, y=0, dw=width, dh=height)



In [42]:
class Streaming(object):    
#     def __init__(self, cam):
    def __init__(self):
#         self.cam = cam
        self.streaming_window = streaming_window
        self.myImage = myImage
        self.stopped = False
        self.t2 = threading.Thread(target=self.update, args=())
        
    def start_thread(self):
        self.t2.start()
        return self
    
    def update(self):
        while True:
            if self.stopped:
                return
            
            try:
#                 image_result = self.cam.GetNextImage()
#                 #  Ensure image completion
#                 if image_result.IsIncomplete():
#                     print('Image incomplete with image status %d ...' % image_result.GetImageStatus())
#                 else:     
                    t1 = time.time()
#                     self.frame = image_result.GetNDArray()
                    self.ret, self.frame = cv2.VideoCapture(0).read()
                    self.frame = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGBA)
                    self.frame = render_bounding_box(self.frame)
                    self.frame = cv2.flip(self.frame, -1)
                    self.frame = image_resize(self.frame, height = 400)
                    self.myImage.data_source.data['image']=[self.frame]
                    t2 = time.time()
                    s = f"""{int(1/(t2-t1))} FPS"""
                    self.streaming_window.title.text = s
                    push_notebook()
#                 image_result.Release()
            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                return False
    
    def read(self):
        return self.frame
    
    def stop(self):
        #self.t2.join()
        self.stopped = True
        self.t2.join()

In [43]:
""" BUTTON CLICK HANDLER """
# def on_start_streaming(cam):
def on_start_streaming():
    t = threading.currentThread()
#     nodemap_tldevice = cam.GetTLDeviceNodeMap()
#     cam.Init()
#     nodemap = cam.GetNodeMap()
#     cam.BeginAcquisition()
#     stream = Streaming(cam).start()
    stream = Streaming().start_thread()
    while getattr(t, "do_run", True):
        time.sleep(0.01)
    stream.stop()
#     cam.EndAcquisition()
#     cam.DeInit()
    print("Stopping as you wish.")

In [44]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    #print(str(future) + "From the function")
    return future

In [5]:
# system = PySpin.System.GetInstance()
# version = system.GetLibraryVersion()
# cam_list = system.GetCameras()
# num_cameras = cam_list.GetSize()

# # This example only works with 1 camera is connected. 
# if num_cameras == 0:
#     # Clear camera list before releasing system
#     cam_list.Clear()

#     # Release system instance
#     system.ReleaseInstance()

#     print('Not enough cameras!')
#     exit()
# elif num_cameras > 1:
#     # Clear camera list before releasing system
#     cam_list.Clear()

#     # Release system instance
#     system.ReleaseInstance()

#     print('This example only works when 1 camera is connected.')
#     exit()

async def f():
    #t = threading.Thread(name= 't', target=on_start_streaming, args=(cam_list[0],))
    t = threading.Thread(name= 't', target=on_start_streaming)
    output_notebook()
    show(streaming_window, notebook_handle=True)
    while True:
        sys.stdout.write('\r')
        sys.stdout.write("Reading Value ")
        sys.stdout.flush()            
        x = await wait_for_change(streaming_toggle, 'value')
        print("Value changed to %s" %x)
        if x == True:
            print("Updating A")
            streaming_toggle.icon = 'stop'
            streaming_toggle.description='Stop Streaming'
            t.start() 
        else:
            t.do_run = False
            t.join() 
            show_init_screen()
            print("Updating B")
            streaming_toggle.icon = 'play'
            streaming_toggle.description='Start Streaming'
            #t = threading.Thread(name= 't', target=on_start_streaming, args=(cam_list[0],))
            t = threading.Thread(name= 't', target=on_start_streaming)

asyncio.ensure_future(f())
display(inference_checkbox)
display(bounding_box_checkbox)
display(inference_mode)
display(streaming_toggle)

Checkbox(value=False, description='Inference')

Checkbox(value=False, description='Bounding Box')

ToggleButtons(description='Mode:', options=('Detection', 'Classification'), tooltips=('This network determines…

ToggleButton(value=False, button_style='info', description='Start Streaming', icon='play', tooltip='Streaming'…

Task exception was never retrieved
future: <Task finished coro=<f() done, defined at <ipython-input-5-5dc5350b1045>:26> exception=NameError("name 'on_start_streaming' is not defined")>
Traceback (most recent call last):
  File "<ipython-input-5-5dc5350b1045>", line 28, in f
    t = threading.Thread(name= 't', target=on_start_streaming)
NameError: name 'on_start_streaming' is not defined
